The following is an attempt to use Google's Tensorflow ML library to run a NN classifier. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import random

voice =pd.read_csv('../input/voice.csv')
voice = pd.DataFrame(voice)
print(voice.head())
# Any results you write to the current directory are saved as output.

Let's get/make more training data. What I do create a new df and slightly edit. I add random noise (a small amount) to the meanfreq variable. Then I append this to the original pd df, making a much larger df. This for loop successively doubles the size of the training data. 

Now that we have imported pd data frame, we need to split into label and feature sets, and split into test and train np arrays.

In [ ]:
for i in range(6):
	copy = voice
	copy['meanfreq']=copy['meanfreq']+random.gauss(.0001,.001) # add noice to mean freq var
	voice=voice.append(copy,ignore_index=True) # make voice df 2x as big
	print("shape of df after {0}th intertion of this loop is {1}".format(i,voice.shape))

In [ ]:
label = voice.pop("label")

# converts from dataframe to np array
voice=voice.values

# convert train labels to one hots
train_labels = pd.get_dummies(label)
# make np array
train_labels = train_labels.values

x_train,x_test,y_train,y_test = train_test_split(voice,train_labels,test_size=0.2)
# # so no we have predictors and y values, separated into test and train

x_train,x_test,y_train,y_test = np.array(x_train,dtype='float32'), np.array(x_test,dtype='float32'),np.array(y_train,dtype='float32'),np.array(y_test,dtype='float32')

We now have training and test data. Let's ensure this all looks good:

In [ ]:
print("x_train looks like:\n",x_train[1:2],"\nand y train looks like:\n",y_train[1:2])


print("shape of x_train is: \n",x_train.shape)

We have data on 20 features, with each set of 20 features predicting male or female voice (in a one hot encoded vector).

Let's do some tensorflow. We need to create placeholders for data we'll feed in, and also placeholders for variables we'll use in our NN.

In [ ]:
# place holder for inputs. feed in later
x = tf.placeholder("float", [None, 20])

# # take 20 features to 1000 nodes in hidden layer. why? just cuz?
w1 = tf.Variable(tf.random_normal([20, 1000],stddev=.5,name='w1'))
# # add biases for each node
b1 = tf.Variable(tf.zeros([1000]))
# calculate activations 
hidden_output = tf.nn.softmax(tf.matmul(x, w1) + b1)

# bring from 10 nodes to 2 for my output
w2 = tf.Variable(tf.random_normal([1000, 2],stddev=.5,name='w2'))

b2 = tf.Variable(tf.zeros([2]))

In [ ]:
# placeholder for correct values 
y_ = tf.placeholder("float", [None,2])
# #implement model. these are predicted ys
y = tf.nn.softmax(tf.matmul(hidden_output, w2) + b2)

In [ ]:
loss = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y, y_, name='xentropy')))

Create an optimizer op, and a train step which minimizes the weight and bias vars. 

In [ ]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_step = opt.minimize(loss, var_list=[w1,b1,w2,b2])

Create a way to assess the accuracy of our prediction:

In [ ]:
tf_correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
tf_accuracy = tf.reduce_mean(tf.cast(tf_correct_prediction, "float"))

Start tf session and initialize variables. 

In [ ]:
def get_mini_batch(x,y):
	rows=np.random.choice(x.shape[0], 100)
	return x[rows], y[rows]

In [ ]:
# start session
sess = tf.Session()
summary_writer = tf.train.SummaryWriter('voices')
# summary_writer = tf.train.SummaryWriter('voices', sess.graph)

# # init all vars
init = tf.initialize_all_variables()
sess.run(init)

Now, it is time to train. Run a loop for ntrials iterations. Feed the training data into the train_step. Then print our accuracy on the test set and the corresponding value of the loss function.

In [ ]:
ntrials = 10000
for i in range(ntrials):
    # get mini batch
    a,b=get_mini_batch(x_train,y_train)
    # run train step, feeding arrays of 100 rows each time
    _, cost =sess.run([train_step,loss], feed_dict={x: a, y_: b})
    if i%100 ==0:
    	print("epoch is {0} and cost is {1}".format(i,cost))

Check accuracy on test set:

In [ ]:
result = sess.run(tf_accuracy, feed_dict={x: x_test, 
                                          y_: y_test})

print("Test accuracy: {}".format(result))

To get a sense of what the predictions look like:

In [ ]:
ans = sess.run(y, feed_dict={x: x_test})
print(y_test[0:3])
print("Correct prediction\n",ans[0:3])

AdamOptimizer and running for 10,000 iterations made big difference in performance. How can we get up to 99,100%?

Hopefully someone else got something out of this, but I am also doing this (primarily) to develop some DS skills. In this kernel, I successfully implemented Tensorflow. Some struggles I had to overcome: 

 - Using pd.getdummies to convert to a one-hot vector.
 - Use .values to convert pd df (which is much easier to manipulate) to a np array (which tensorflow reads).
 - Ensure np.array dtype.
 - Use cross_entropy_with_logits to deal with outputs close to 1 and 0.
 - Create function to extract random sample for a mini_batch for NN training.
 - Getting accustomed to TF's idea of placeholders and computational graphs.

 